<h1>pytorch 演習</h1>
<h2>numpyを使用したニューラルネットワーク</h2>

In [51]:
import numpy as np
# N : バッチサイズ
# D_in : 入力次元数
# H : 隠れ層の次元数
# D_out : 出力次元数
N, D_in, H, D_out = 64, 1000, 100, 10
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # forward pass : yを計算
    h = x.dot(w1)
    h_relu = np.maximum(h,0)
    y_pred = h_relu.dot(w2)
    
    # lossを計算して出力
    loss = np.square(y_pred - y).sum()
    #print(t,loss)
    
    # 誤差逆伝播(勾配の計算)
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    #重みの更新
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>そのまま実装</h4>

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")
#device = torcn.device("cuda:0")

# N : バッチサイズ
# D_in : 入力次元数
# H : 隠れ層の次元数
# D_out : 出力次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 入出力データをランダムに生成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 重みをランダムに初期化
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass : 予測値yの計算
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # lossの計算と出力
    loss = (y_pred - y).pow(2).sum().item()
    #print(t,loss)
    
    # 誤差逆伝播（勾配の計算）
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    ##重みの更新
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>自動微分実装</h4>

In [19]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # lossの計算と出力
    loss = (y_pred - y).pow(2).sum()
    #print(t, loss.item())
    
    # 自動で微分計算 -> 誤差逆伝播
    loss.backward()
    
    # 手動で重み更新
    # torch.optim.SGDを使用しても良い
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        # 重み更新後、手動で勾配をゼロにする
        w1.grad.zero_()
        w2.grad.zero_()
    

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>自動微分 関数の定義</h4>

In [27]:
import torch

class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, =ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
    
dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    relu = MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    #print(t, loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>NNモジュール</h4>

In [32]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# モデルを一連のレイヤーとして定義する
# nn.Sequentialは他のモジュールを含むモジュール
# 各線形モジュールは、入力使って出力を計算する
# 線形関数であり、重みのバイアスのために内部テンソルを保持する
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
# nnパッケージには損失関数の定義も含まれている
# 平均二乗誤差を使用するケース
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate=1e-4
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    #print(t, loss.item())
    
    model.zero_grad()
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>optimモジュール</h4>

In [36]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    #print(t, loss.item())
    
    # backwardの前に勾配・重みをゼロにする
    # backwardが実行されるたびに勾配がバッファに蓄積される
    # このせいで上書きができないため、zero_grad()を実行
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>カスタムnnモジュール</h4>

In [39]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
model = TwoLayerNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    #print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

<h2>pytorchを使用したニューラルネットワーク</h2>
<h4>コントロールフロー、重み分割 ← ? </h4>

In [42]:
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    #print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()